In [1]:
from urllib.request import urlopen
from urllib.parse import quote_plus # 아스키 코드로 변환시켜준다.
import urllib.request
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException
from requests.exceptions import ConnectionError
import unicodedata
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import os
import urllib.request        
import filecmp
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from glob import glob

In [2]:
MONTH_STR_TO_INT = {
    'January' : 1,
    'February' : 2,
    'March' : 3,
    'April' : 4,
    'May' : 5,
    'June' : 6,
    'July' : 7,
    'August' : 8,
    'September' : 9,
    'October' : 10,
    'November' : 11,
    'December' : 12
    
}

In [3]:
def isKorean(text):
    hangul = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')  
    result = hangul.findall(text)
    return len(result)

In [4]:
def upload_date_check():
    html = driver.page_source
    soup = BeautifulSoup(html)
    # driver.find_element_by_xpath('.//time [@class = "_aaqe"]').text# datetime
    # time.sleep(1)
    

    for idx, i in enumerate(soup.findAll('time')):
        if i.has_attr('datetime'):
            TIME = i['datetime']

            p = re.search('(\d+)-(\d+)-(\d+)', TIME)
            if p:
                yyyy, mm, dd = [int(x) for x in p.groups()]
                print(datetime.datetime.now().date() - datetime.date(yyyy, mm, dd), end='\r')
                return yyyy, mm, dd
        
    return False

In [5]:
def crawl_influencers():
    driver.find_element_by_partial_link_text("팔로우").click()
    #driver.find_element_by_css_selector('a').click()
    time.sleep(5)

    SCROLL_PAUSE = 0.5  # Pause to allow loading of content
    driver.execute_script("followingbox = document.getElementsByClassName('_aano')[0];")
    last_height = driver.execute_script("return followingbox.scrollHeight;")

    # We need to scroll the following modal to ensure that all following are loaded
    while True:
        driver.execute_script("followingbox.scrollTo(0, followingbox.scrollHeight);")

        # Wait for page to load
        time.sleep(SCROLL_PAUSE)

        # Calculate new scrollHeight and compare with the previous
        new_height = driver.execute_script("return followingbox.scrollHeight;")
        if new_height == last_height:
            time.sleep(3)
            driver.execute_script("followingbox.scrollTo(0, followingbox.scrollHeight);")
            if new_height == last_height:
                break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    followings = soup.findAll('a',['FPmhX','notranslate','_0imsa'])
    followings_text = []
    for following in followings:
        followings_text.append(following.get_text())
            
    rest= requests.get(url)
    soup =BeautifulSoup(rest.content, 'html.parser')
    
    close_following_bttn = driver.find_element_by_class_name('_ac7b._ac7d')
    close_following_bttn.click()

    influencer_following = [ x.split('인증됨')[0] for x in followings_text if len(x.split('인증됨'))==2 ]
    
    return influencer_following

In [6]:
def image_save(SRC, PATH, UPLOAD_DATE):
    yyyy, mm, dd = UPLOAD_DATE
    exists_files = glob(f'{download_directory}/{datetime.date(yyyy, mm, dd)}*')
    with open(f'{PATH}/{datetime.date(yyyy, mm, dd)}_{len(exists_files)+1}.jpg', 'wb') as handler: 
        handler.write(SRC)

In [7]:
personal_info = pd.read_csv('personal_info.csv')
ID, PW = personal_info['ID'], personal_info['PW']

In [8]:
influencers = pd.read_csv('new_influencer.csv', index_col=0)
# influencers = [x.strip() for x in influencers['influencer'].values]
exists_df = influencers.index.isin(os.listdir('./img'))
influencers = influencers[~exists_df]

In [9]:
baseUrl = 'https://www.instagram.com/'
plusUrl = influencers.index[0]# 'jin_a_nana'# influencers['influencer'][9].strip()
url = baseUrl + quote_plus(plusUrl)

print(url)

https://www.instagram.com/nizinanjjang


In [10]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(baseUrl) # 드라이버를 띄운다. (크롬 웹 페이지를 연다.)
driver.implicitly_wait(5)

driver.find_element_by_name('username').send_keys(ID)
driver.implicitly_wait(5)
elem_pw = driver.find_element_by_name('password') # .submit()
elem_pw.send_keys(PW)
driver.implicitly_wait(5)
elem_pw.submit()
driver.implicitly_wait(5)
time.sleep(5)

In [11]:
influencers.index[0]

'nizinanjjang'

In [12]:
baseUrl = 'https://www.instagram.com/'
plusUrl = influencers.index[0]# 'jin_a_nana'# influencers['influencer'][9].strip()
url = baseUrl + quote_plus(plusUrl)
download_directory = os.path.join('./img', plusUrl)

for plusUrl in influencers.index[:10]:
    url = baseUrl + quote_plus(plusUrl)
    print(url)

https://www.instagram.com/nizinanjjang
https://www.instagram.com/davikah
https://www.instagram.com/collagevintage2
https://www.instagram.com/songdani
https://www.instagram.com/carodaur
https://www.instagram.com/stylebop
https://www.instagram.com/hikari
https://www.instagram.com/korea_grandma
https://www.instagram.com/yoyokulala
https://www.instagram.com/thesartorialist


In [13]:
influencers.index[0]

'nizinanjjang'

In [14]:
# influencers = influencers.drop(influencers.index[0])

# influencers.to_csv('new_influencer.csv')

In [15]:
influencers = pd.read_csv('new_influencer.csv', index_col=0)

In [ ]:
# for plusUrl in influencers[len(os.listdir(r'C:\Users\anapl\CDAL\fashion\instagram-crawler\img')):]:
while True:    
    influencers = pd.read_csv('new_influencer.csv', index_col=0)
    exists_df = influencers.index.isin(os.listdir('./img'))
    influencers = influencers[~exists_df]    
    plusUrl = influencers.index[0] # 'happpy_yein'
    url = baseUrl + quote_plus(plusUrl)
    try:
        driver.get(url)
    except CannotSendRequest:
        continue
    time.sleep(3)
    
    rest = requests.get(url)
    soup = BeautifulSoup(rest.content, 'html.parser')
    
    """if soup.find('meta', {'name': 'description'}) == None:
        print("\nSOUP NO FOUND")
        influencers = influencers.drop(influencers.index[0])
        influencers.to_csv('new_influencer.csv')
        continue"""
        
    # 이름 확인
    try:
        RAW_NAME = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/section/main/div/header/section/div[3]/span').text
        NAME = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/section/main/div/header/section/div[3]/span').text
        NAME = unicodedata.normalize('NFC', NAME)
    except NoSuchElementException:
        print(f"\nNO NAME {len(influencers)}")
        influencers = influencers.drop(influencers.index[0])
        influencers.to_csv('new_influencer.csv')
        continue
    
    if isKorean(NAME):
        pass
    else:
        print(f"\nNO KOREAN {len(influencers)}")
        influencers = influencers.drop(influencers.index[0])
        influencers.to_csv('new_influencer.csv')
        continue


    time.sleep(1)
    try:
        influencer_following = crawl_influencers()
        insta_link = ['https://www.instagram.com/'+x+'/' for x in influencer_following]
        df = pd.DataFrame([influencer_following, insta_link], index=['influencer', 'instagram']).T
        df_inf = pd.read_csv('new_influencer.csv')
        new_df = pd.concat([df_inf, df]).reset_index(drop=True)
        new_df.to_csv("new_influencer.csv", index=False)
    except NoSuchElementException:
        print(f"\nNO FOLLOWING {len(influencers)}")

    
    body = driver.find_element_by_css_selector('body')
    download_directory = os.path.join('./img', plusUrl)

    html = driver.page_source
    soup = BeautifulSoup(html)
        
    if not os.path.isdir('./img'):
        os.mkdir('./img')

    if os.path.isdir(os.path.join('./img', plusUrl)):
        print(f"{plusUrl} Directory Exists")
        if len(os.listdir(os.path.join('./img', plusUrl))) != 0:
            continue
    else:
        os.mkdir(os.path.join('./img', plusUrl))
        print(f"{plusUrl} Make Directory")

    # 첫번째 이미지 클릭
    btn_images = driver.find_elements_by_class_name('_aabd._aa8k._aanf')

    btn = btn_images[0]
    btn.click()
    time.sleep(2)

    downloaded_image_list = []
    global_stack = 0
    while True:
        image_dict = {}
        exists = os.listdir(download_directory)
        while True:
            if upload_date_check() != False:
                yyyy, mm, dd = upload_date_check()
                upload_date = (yyyy, mm, dd)
                time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)
            else:
                try:
                    btn_next =  driver.find_element_by_class_name('_9zm2')
                    btn_next.click()
                except NoSuchElementException:
                    try:
                        next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                        next_post.click()
                    except NoSuchElementException:
                        pass
                    finally:
                        break

            for x in driver.find_elements_by_xpath('//div/img')[-3:]:
                if x.get_property('alt').startswith('Photo'): 
                    print(x.get_property('alt'))
                    print(x.get_attribute('src'))
                    UPLOAD_DATE_IN_STR = x.get_property('alt').split('May be an image')[0].strip().split(RAW_NAME)[-1].strip()#startswith(f'Photo by {RAW_NAME} on')
                    p = re.search('\w+ (\w+) (\d+), (\d+)', UPLOAD_DATE_IN_STR)
                    if p is None:
                        continue
                    mm, dd, yyyy = MONTH_STR_TO_INT[p.groups()[0]], p.groups()[1], p.groups()[2]
                    mm, dd, yyyy = int(mm), int(dd), int(yyyy)
                    if str(datetime.date(yyyy, mm, dd)) not in image_dict.keys(): 
                        image_dict[x.get_attribute('src')] = str(datetime.date(yyyy, mm, dd))
            try:
                btn_next =  driver.find_element_by_class_name('_9zm2')
                btn_next.click()
            except NoSuchElementException:
                try:
                    next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                    next_post.click()
                except NoSuchElementException:
                    pass
                finally:
                    break
            except StaleElementReferenceException:
                try:
                    next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                    next_post.click()
                except NoSuchElementException:
                    pass
                finally:
                    break
            finally:
                time.sleep(1)

        # image_list = list(set(image_dict.keys))

        for source in list(image_dict.keys()):
            upload_date = image_dict[source]
            src = requests.get(source).content
            time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)
            if source not in downloaded_image_list and time_delta < datetime.timedelta(days=90):
                image_save(src, download_directory, (yyyy, mm, dd))
                time.sleep(0.5)
            downloaded_image_list.append(source)
            # downloaded_image_list = downloaded_image_list + image_list
            time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)
        try:
            time_delta
        except NameError:
            continue
        if time_delta > datetime.timedelta(days=90):
            break
    print()
    btn_close = driver.find_element_by_class_name('x78zum5.x6s0dn4.xl56j7k.xdt5ytf')
    btn_close.click()

nizinanjjang Make Directory
Photo shared by nanjeong Lee 이난정 on July 07, 2021 tagging @tabithasimmons, @matchesfashion, and @tannerkrolle.
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/211483641_216490953674258_6632226716910889316_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=zErZsL_g5TIAX-Axmme&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MjYxMjYwNDQ2MDU3MjQyNTQ5NQ%3D%3D.2-ccb7-5&oh=00_AfD8vZUFhkmbv5sCumRnIqkcH3Mzyfjm1zOfAngk8d8VTQ&oe=637EE416&_nc_sid=30a2ef
Photo by nanjeong Lee 이난정 on July 07, 2021.
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/210835504_1212880919160392_1729989204473320470_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=7CEqqIMnAh0AX9fa2De&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MjYxMjYwNDQ2MDUzMDQ5NDAwMA%3D%3D.2-ccb7-5&oh=00_AfCXcS29GhlsynRPNsnCrGAgYFNppou8y7pDzyN7Sunm5Q&oe=637F5027&_nc_sid=30a2ef
Photo shared by nanjeong Lee 이난정 on July 07, 2021 tagging @tabithasimmons, 

In [ ]:
rest = requests.get(url)
soup = BeautifulSoup(rest.content, 'html.parser')

In [ ]:
for x in driver.find_elements_by_xpath('//div/img')[-3:]:
    if x.get_property('alt').startswith('Photo'): 
        print(x.get_property('alt'))
        print(x.get_attribute('src'))
        UPLOAD_DATE_IN_STR = x.get_property('alt').split('May be')[0].strip().split(RAW_NAME)[-1].strip()#startswith(f'Photo by {RAW_NAME} on')
        p = re.search('\w+ (\w+) (\d+), (\d+)', UPLOAD_DATE_IN_STR)
        if p is None:
            continue
        mm, dd, yyyy = MONTH_STR_TO_INT[p.groups()[0]], p.groups()[1], p.groups()[2]
        mm, dd, yyyy = int(mm), int(dd), int(yyyy)
        print(str(datetime.date(yyyy, mm, dd)))
        if str(datetime.date(yyyy, mm, dd)) not in image_dict.keys(): 
            image_dict[x.get_attribute('src')] = str(datetime.date(yyyy, mm, dd))

In [ ]:
UPLOAD_DATE_IN_STR

In [ ]:
list(image_dict.keys())

In [ ]:
upload_date

In [ ]:
p = re.search('(\d+)-(\d+)-(\d+)', upload_date)
if p:
    yyyy, mm, dd = [int(x) for x in p.groups()]

In [ ]:
datetime.datetime.strptime(upload_date, '%Y-%m-%d').date()

In [161]:
print(list(image_dict.keys()))
for src in list(image_dict.keys()):
    upload_date = image_dict[src]
    src = requests.get(src).content
    image_save(src, download_directory, (yyyy, mm, dd))
    time.sleep(1)

['https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7', 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/314915133_637555451435380_8898475586681211251_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=vp13XJbHyacAX8y_412&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjk2OTU5OTIzNjA1MzY1MzQ5Nw%3D%3D.2-ccb7-5&oh=00_AfApGfxY5HnsKGuVCZsvI5tcqDhsnboRs4NYdA2BIHFGAA&oe=637A3913&_nc_sid=30a2ef']


In [131]:
image_dict.keys()

dict_keys([])

In [126]:
str(datetime.date(yyyy, mm, dd))

'2022-11-01'

In [171]:
f'Photo by {NAME}'

'Photo by 옥주현'

In [186]:
RAW_NAME = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/section/main/div/header/section/div[3]/span').text
UPLOAD_DATE_IN_STR = x.get_property('alt').split('May be an image')[0].strip().split(RAW_NAME)[-1].strip()#startswith(f'Photo by {RAW_NAME} on')

In [187]:
UPLOAD_DATE_IN_STR

'on October 25, 2022.'

In [195]:
MONTH_STR_TO_INT = {
    'January' : 1,
    'February' : 2,
    'March' : 3,
    'April' : 4,
    'May' : 5,
    'June' : 6,
    'July' : 7,
    'August' : 8,
    'September' : 9,
    'October' : 10,
    'November' : 11,
    'December' : 12
    
}

In [200]:
p.groups()[0]

'October'

In [202]:
p = re.search('\w+ (\w+) (\d+), (\d+)', UPLOAD_DATE_IN_STR)
print(p.groups())
mm, dd, yyyy = MONTH_STR_TO_INT[p.groups()[0]], p.groups()[1], p.groups()[2]
print(mm, dd, yyyy)

('October', '25', '2022')
10 25 2022


In [178]:
for x in driver.find_elements_by_xpath('//div/img')[-3:]:
    print(x.get_property('alt'))
    if x.get_property('alt').startswith('Photo'): 
        print(x.get_attribute('src'))
        image_dict = x.get_attribute('src')

Photo by 옥주현 on September 10, 2022. May be an image of 1 person, flower and indoor.
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7
Photo by 옥주현 on October 25, 2022. May be an image of flower and text that says '프란츠 요제프 1세의 도시 확장 정책 수도 빈은 프란츠 요제프 1세의 도시 확장 정책으로 문화와 예술의 중심지이자 유럽의 대표적인 관광 명소로 자리 잡았습니다 빈은 외부로부터 수도를 보호하기 위해 지은 성벽으로 둘러싸여 있었습니다. 1857년 황제의 명령으로 빈을 둘러 성벽이 철거되었고 그 자리에 반지 모양의 도로 링슈트라세가 들어섰습니다. 링슈트라세를 따라 이제는 도시 빈을 대표하는 명소로 자리 잡은 시청, 국회의사당, 대학, 국립극장, 오페라극장 그리고 빈미술사박물관과 빈자연사박물관이 건축되었습니다.'.
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/312443553_168980799066481_861376557339861004_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_n

In [113]:
image_list

['https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7',
 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/312996840_1287448498465627_5859498371585224184_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=107&_nc_ohc=2odZ_TXxIZoAX9-e8qv&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjk2MjAyNjc3NjIyNzY0MTQ3NA%3D%3D.2-ccb7-5&oh=00_AfDx6BU73UTuymYO9h38FakvporNcJ93kw1THv-q_GmcIQ&oe=637AB565&_nc_sid=30a2ef',
 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/313495428_570195221774472_3211005464460916054_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=100&_nc_ohc=iYq2gWb39osAX_JxRhW&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjk2MjAyNjc3NjM5NTUwMj

In [99]:
for x in driver.find_elements_by_xpath('//div/img')[-3:]:
    if x.get_property('alt').startswith('Photo'): 
        print(x.get_attribute('src'))
        image_list.append(x.get_attribute('src'))

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/313689192_5565705930165340_7470918217344741727_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=JDH60kAM2HwAX_AkYSa&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjk2MzIyNTEyMTY1NDY3NzE3OQ%3D%3D.2-ccb7-5&oh=00_AfBEjHtafk38XqN7OIPTHqz93CGqKfFK6LpXT7M3zsADcQ&oe=637A5503&_nc_sid=30a2ef
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/313944671_101378119442903_7828911811475646877_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=3sx439CKvScAX-p6JwX&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjk2MzIyNTEyMTY1NDY1MjM4Mg%3D%3D

In [111]:
for src in image_list:
    print(src)
    if type(src) == str:
        src = requests.get(src).content
    
    # if src not in downloaded_image_list:
    image_save(src, download_directory, upload_date)
    time.sleep(0.5)
    downloaded_image_list = downloaded_image_list + image_list
    time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/313689192_5565705930165340_7470918217344741727_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=JDH60kAM2HwAX_AkYSa&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjk2MzIyNTEyMTY1NDY3NzE3OQ%3D%3D

In [96]:
len(downloaded_image_list)

2

In [94]:
image_list

['https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7']

In [ ]:
yyyy

In [23]:
download_directory = os.path.join('./img', plusUrl)

In [24]:
download_directory

'./img\\nayoungkeem'

In [45]:
influencers = pd.read_csv('new_influencer.csv', index_col=0)
exists_df = influencers.index.isin(os.listdir('./img'))
influencers = influencers[~exists_df]    
plusUrl = 'o._.julia'# influencers.index[0] # 'happpy_yein'
url = baseUrl + quote_plus(plusUrl)
try:
    driver.get(url)
except CannotSendRequest:
    # continue
    pass
time.sleep(3)

In [47]:
rest = requests.get(url)
soup = BeautifulSoup(rest.content, 'html.parser')

if soup.find('meta', {'name': 'description'}) == None:
    print("\nSOUP NO FOUND")
    influencers = influencers.drop(influencers.index[0])
    influencers.to_csv('new_influencer.csv')
    pass

In [49]:
# 이름 확인
try:
    NAME = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/section/main/div/header/section/div[3]/span').text
    NAME = unicodedata.normalize('NFC', NAME)
except NoSuchElementException:
    print(f"\nNO NAME {len(influencers)}")
    influencers = influencers.drop(influencers.index[0])
    influencers.to_csv('new_influencer.csv')
    pass

In [51]:
if isKorean(NAME):
    pass
else:
    print(f"\nNO KOREAN {len(influencers)}")
    influencers = influencers.drop(influencers.index[0])
    influencers.to_csv('new_influencer.csv')
    pass

In [52]:
summary_content = soup.find('meta', {'name': 'description'})['content']
follower_count = summary_content.split(' ')[0]
# print("팔로워 수: "+str(follower_count))
following_count = summary_content.split(' ')[2]
# print("팔로잉 수: "+str(following_count))
post_count = summary_content.split(' ')[4]
# print("게시물 수: "+str(post_count))

In [53]:
time.sleep(1)
try:
    influencer_following, follower_count = crawl_influencers()
    insta_link = ['https://www.instagram.com/'+x+'/' for x in influencer_following]
    df = pd.DataFrame([influencer_following, insta_link], index=['influencer', 'instagram']).T
    df_inf = pd.read_csv('new_influencer.csv')
    new_df = pd.concat([df_inf, df]).reset_index(drop=True)
    new_df.to_csv("new_influencer.csv", index=False)
except NoSuchElementException:
    print(f"\nNO FOLLOWING {len(influencers)}")

In [54]:
body = driver.find_element_by_css_selector('body')
download_directory = os.path.join('./img', plusUrl)

html = driver.page_source
soup = BeautifulSoup(html)

In [56]:
if not os.path.isdir('./img'):
    os.mkdir('./img')

if os.path.isdir(os.path.join('./img', plusUrl)):
    print(f"{plusUrl} Directory Exists")
    if len(os.listdir(os.path.join('./img', plusUrl))) != 0:
        pass
else:
    os.mkdir(os.path.join('./img', plusUrl))
    print(f"{plusUrl} Make Directory")

o._.julia Directory Exists


In [57]:
# 첫번째 이미지 클릭
btn_images = driver.find_elements_by_class_name('_aabd._aa8k._aanf')

btn = btn_images[0]
btn.click()
time.sleep(2)

In [89]:
downloaded_image_list = []
global_stack = 0
while True:
    image_list = []
    exists = os.listdir(download_directory)
    while True:
        if upload_date_check() != False:
            yyyy, mm, dd = upload_date_check()
            upload_date = (yyyy, mm, dd)
            time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)
        else:
            try:
                btn_next =  driver.find_element_by_class_name('_9zm2')
                btn_next.click()
            except NoSuchElementException:
                try:
                    next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                    next_post.click()
                except NoSuchElementException:
                    pass
                finally:
                    break

        if time_delta > datetime.timedelta(days=90):
            break

        for x in driver.find_elements_by_xpath('//div/img')[-5:]:
            if x.get_property('alt').startswith('Photo'): 
                print(x.get_attribute('src'))
                image_list.append(x.get_attribute('src'))
        try:
            btn_next =  driver.find_element_by_class_name('_9zm2')
            btn_next.click()
        except NoSuchElementException:
            try:
                next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                next_post.click()
            except NoSuchElementException:
                pass
            finally:
                break
        except StaleElementReferenceException:
            try:
                next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                next_post.click()
            except NoSuchElementException:
                pass
            finally:
                break
        finally:
            time.sleep(1)

    image_list = list(set(image_list))
    try:
        images = [requests.get(src).content for src in image_list]
    except ConnectionError:
        body.send_keys(Keys.ARROW_RIGHT)
        time.sleep(5)
        continue
    for src in image_list:
        if src not in downloaded_image_list:
            image = requests.get(src).content
            image_save(image, download_directory, upload_date)

            downloaded_image_list = downloaded_image_list + image_list
            time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)
    try:
        time_delta
    except NameError:
        continue
    if time_delta > datetime.timedelta(days=90):
        break
print()

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2-ccb7-5&oh=00_AfByUDKSXkSFkSh05OLGbYON2HAtDBQFIuaX981_NERiSA&oe=637ADFDD&_nc_sid=6136e7
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/312412312_118497357699051_696758424604402565_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=MuA5wNldTtYAX_b5Fqu&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjk1NTQ1OTY3OTUwNzQwODY5MQ%3D%3D.2-ccb7-5&oh=00_AfCfTEabfY710boNpsyzV97riiDqVWnPk_p1hxZcGFZTGA&oe=637B96B8&_nc_sid=30a2ef
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/306382483_799968194369269_4880640767374534227_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=qFgg3eYe4dMAX-rYw-7&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MjkyNDEyNzUyNjk3OTA3MDY3OQ%3D%3D.2

KeyboardInterrupt: 